In [1]:
import sys, os
import json
import pandas as pd
import numpy as np
from IPython.display import Markdown, display
import ipywidgets as widgets
import plotly
import plotly.graph_objects as graph_obj
import plotly.figure_factory as ff

sys.path.append(os.path.join(os.getcwd(),'../utils'))
from s3 import s3store
from jsonutil import WriteDictJson, ReadDictJson

In [2]:
output = widgets.Output()
clear_output = widgets.Button(description='Clear Output')

test_select = widgets.Select(
    options=[],
    description='Test:',
    disabled=False,
    #rows=25,
)

In [3]:
trainingset='coco'

In [4]:
creds = ReadDictJson('../creds.json')
if not creds:
    output.append_stdout('Failed to load credentials file {}\n'.format(args.credentails))

s3def = creds['s3'][0]
s3 = s3store(s3def['address'], s3def['access key'], s3def['secret key'], cert_verify=False)
validation = s3def['sets']['trainingset']

test_obj = '{}/{}/tests.json'.format(validation['prefix'], trainingset)
desc_obj = '{}/{}/description.json'.format(validation['prefix'], trainingset)
testjson = s3.GetDict(validation['bucket'], test_obj)
descjson = s3.GetDict(validation['bucket'], desc_obj)

tests = pd.DataFrame(testjson)
display(tests)

,date,model,accuracy,class_similarity,similarity,confusion,images,image time,batch size,test store,test bucket,results,name,description,store address,platform
0,"04/27/2021, 17:44:13",2021-02-24-10-28-35-cocoseg,0.947308,"{'0': {'intersection': 0, 'union': 0, 'similar...",0.668131,"[[1019645833, 9687226, 4062913, 2917721], [192...",5000,0.079917,1,ipc.larson.myds.me:31994,mllib,"{'class similarity': {'0': {'intersection': 0,...",NaN,NaN,NaN,NaN
1,"04/27/2021, 20:14:53",2021-02-24-10-28-35-cocoseg,0.946964,"{'0': {'intersection': 0, 'union': 0, 'similar...",0.666706,"[[1019448998, 9814403, 4245050, 2854714], [194...",5000,0.008515,1,NaN,mllib,NaN,Test TensorRT inference,TensorRT image segmentation inference test,ipc.larson.myds.me:31994,Linux-4.15.0-142-generic-x86_64-with-glibc2.29
2,"04/27/2021, 22:13:24",2021-02-24-10-28-35-cocoseg,0.942014,"{'0': {'intersection': 0, 'union': 0, 'similar...",0.666543,"[[1019283446, 9913144, 4228162, 2780527], [194...",5000,0.047200,1,NaN,mllib,NaN,Test TensorRT inference,TensorRT image segmentation inference test,ipc:31994,Linux-4.9.201-tegra-aarch64-with-Ubuntu-18.04-...
3,"04/27/2021, 23:15:45",2021-02-24-10-28-35-cocoseg,0.945001,"{'0': {'intersection': 0, 'union': 0, 'similar...",0.667415,"[[1019324437, 9787572, 4121237, 2904945], [191...",5000,0.029516,1,NaN,mllib,NaN,Test TensorRT inference,TensorRT image segmentation inference test,192.168.0.155:31994,Linux-4.9.201-tegra-aarch64-with-Ubuntu-18.04-...


In [5]:
iClasses = {}
classNames = [''] * descjson['classes']['classes']
for clasdesc in descjson['classes']['objects']:
    if clasdesc['trainId'] not in iClasses.keys():
        classNames[clasdesc['trainId']] = clasdesc['category']
        if not (clasdesc['category'] == 'void'):
            iClasses[clasdesc['trainId']] = clasdesc

overview = {}
results = {}
confusion = {}
test_names = []
    
for test in testjson:
    if 'name' in test:
        name = '{} {}'.format(test['name'], test['date'])
    elif 'model' in test:
        name = '{} {}'.format(test['model'], test['date'])
    else: 
        name = '{}'.format(test['date'])
    test_names.append(name)
    
    test_overview = {'images':test['images'],'accuracy':test['accuracy'],'similarity':test['similarity'], 'inference time':test['image time']}
    overview[name]=test_overview
    
    similarity = {}
    for key in iClasses:
        res = test['class_similarity'][str(key)]
        if res:
            similarity[iClasses[key]['category']] = test['class_similarity'][str(key)]['similarity']

    results[name] = {'similarity':similarity, 'confusion':test['confusion']}
    
display(pd.DataFrame(overview).T)

,images,accuracy,similarity,inference time
"2021-02-24-10-28-35-cocoseg 04/27/2021, 17:44:13",5000.0,0.947308,0.668131,0.079917
"Test TensorRT inference 04/27/2021, 20:14:53",5000.0,0.946964,0.666706,0.008515
"Test TensorRT inference 04/27/2021, 22:13:24",5000.0,0.942014,0.666543,0.047200
"Test TensorRT inference 04/27/2021, 23:15:45",5000.0,0.945001,0.667415,0.029516


In [6]:
c = np.array(results[test_names[0]]['confusion'])
norm_confusion = (c.T / c.astype(np.float).sum(axis=1)).T
norm_confusion = norm_confusion.round(decimals=3)

confusion_text = [[str(round(y, 3)) for y in x] for x in norm_confusion]

# set up figure 
confusion_plot = plotly.figure_factory.create_annotated_heatmap(norm_confusion, x=classNames, y=classNames, colorscale='plasma')

# add custom xaxis title
confusion_plot.add_annotation(dict(font=dict(color="black",size=16),
                   x=0.5,
                   y=-0.15,
                   showarrow=False,
                   text="Predicted value",
                   xref="paper",
                   yref="paper"))

# add custom yaxis title
confusion_plot.add_annotation(dict(font=dict(color="black",size=16),
                   x=-0.2,
                   y=0.5,
                   showarrow=False,
                   text="Real value",
                   textangle=-90,
                   xref="paper",
                   yref="paper"))


# adjust margins to make room for yaxis title
confusion_plot.update_layout(margin=dict(t=50, l=200))

# add colorbar
confusion_plot['data'][0]['showscale'] = True
confusion_plot.update_layout(yaxis_autorange="reversed")

confusion_display = graph_obj.FigureWidget(confusion_plot)


In [7]:
def SelectTest(change, select, display, testjson):
    global test_select
    global confusion_display
    global output
    global results
    global confusion_plot
    
    with confusion_display.batch_update():
        
        c = np.array(results[test_select.value]['confusion'])
        norm_confusion = (c.T / c.astype(np.float).sum(axis=1)).T
        confusion_text = [str(round(x, 3)) for x in norm_confusion.flatten()]
        confusion_display.data[0].z =  norm_confusion
        confusion_display.update_layout(yaxis_autorange="reversed")
        
        for i in range(len(confusion_text)):
            confusion_display.layout.annotations[i].text = confusion_text[i]
            
def ClearOutput(b):
    global test_select
    global confusion_display
    global output
    output.clear_output()

In [8]:

clear_output.on_click(lambda b: ClearOutput(b))


display_results = widgets.HBox([test_select, confusion_display])
attach = widgets.VBox([display_results, clear_output, output])

test_select.observe (lambda change:SelectTest(change, select=test_select, display=confusion_display, testjson=testjson), names="value")

if len(test_names) > 0:
    test_select.options = test_names
    test_select.value = test_names[0]

display(attach)